In [76]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

# Path of our data
path = r"C:\Users\Zakaria-Laptop\FirstStepWithGit\data\customer_data.csv"

In [46]:
def read_data(path: str)-> pd.DataFrame:
    return pd.read_csv(path)

df = read_data(path)
df.head(2)

,CustomerID,Name,Gender,Age,Email,JoinDate,PurchaseAmount,LastLogin
0,1001,Customer 1,NaN,65.0,customer1@example.com,2020-07-23,909.33,2024-03-14
1,1002,Customer 2,NaN,18.0,customer2@example.com,NaN,771.89,NaN


## La base de données

- CustomerID : Identifiant unique du client
- Name : Nom du client
- Gender : Genre du client (M, F, null)
- Age : Âge du client (peut contenir des valeurs aberrantes ou manquantes)
- Email : Adresse e-mail du client
- JoinDate : Date d'inscription au service (peut contenir des formats incorrects)
- PurchaseAmount : Montant des achats (peut contenir des valeurs manquantes ou incorrectes)
- LastLogin : Dernière date de connexion (peut contenir des valeurs nulles)

## Niveau - 1: Basique

In [47]:
# Combien de lignes et de colonnes contient la base de données
print(f"le nombre de lignes dans cette base de données est: {df.shape[0]} et le nombre des colonnes est: {df.shape[1]}")

# Identifie s'il y'a des valeurs manquantes dans chaque colonne
print("les valeurs manquantes de chaques colonnes: \n", df.isna().sum())

# Gestion des valeurs manquantes

# 1- Remplacez les valeurs manquantes de la colonne "Gender" par "Unknown"
df["Gender"].fillna("Unknown", inplace=True)
# 2- Remplacez Nan sur Age par la médiane des ages
df["Age"].fillna(df["Age"].median(), inplace=True)
# 3- Remplissez les Nans de "PurchaseAmount" par la moyenne.
df["PurchaseAmount"].fillna(df["PurchaseAmount"].median(), inplace=True)

# Vérifier les modifications
print("\nAprès remplacement des valeurs manquantes :")
print(df.isna().sum())

le nombre de lignes dans cette base de données est: 50 et le nombre des colonnes est: 8
les valeurs manquantes de chaques colonnes: 
 CustomerID         0
Name               0
Gender            14
Age               12
Email              0
JoinDate          20
PurchaseAmount    18
LastLogin         29
dtype: int64

Après remplacement des valeurs manquantes :
CustomerID         0
Name               0
Gender             0
Age                0
Email              0
JoinDate          20
PurchaseAmount     0
LastLogin         29
dtype: int64


## Niveau 2: Intermédiaire


In [84]:
# 1 - Suppression des doublons
print("Le nombre des doublons dans la colonne 'CustomerID' est: ", df["CustomerID"].duplicated().sum())
df.drop_duplicates(subset="CustomerID", inplace=True)

# 2 - Formatage des colonnes
# Assurez que la colonne "JoinDate" soit bien au format datetime
df["JoinDate"] = pd.to_datetime(df["JoinDate"], errors="coerce")

# Convertis la colonne "LastLogin" en datetime et remplace Nans par la date la plus ancienne.
df["LastLogin"] = pd.to_datetime(df["LastLogin"], errors="coerce")
last_date = df["LastLogin"].min()
df["LastLogin"].fillna(last_date, inplace=True)

# Correction des erreurs dans les données
# 1- Identifie et supprime les valeurs aberrantes dans la colonne "Age" (ex. : âges négatifs ou trop élevés).

Q1 = df["Age"].quantile(0.25)
Q3 = df["Age"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5*IQR
upper_bound = Q3 + 1.5*IQR

print(f"Lower_bound is equal: {lower_bound} and our upper bound is: {upper_bound}")
df = df[(df["Age"]>lower_bound)&(df["Age"]<upper_bound)]

# Normaliser les montants d'achat : rendre tous les montants négatifs positifs
df['PurchaseAmount'] = df['PurchaseAmount'].abs()

print("\nValeurs aberrantes corrigées dans 'Age' et 'PurchaseAmount' :")
print(df[['Age', 'PurchaseAmount']].describe())

Le nombre des doublons dans la colonne 'CustomerID' est:  0
Lower_bound is equal: -18.25 and our upper bound is: 105.75

Valeurs aberrantes corrigées dans 'Age' et 'PurchaseAmount' :
             Age  PurchaseAmount
count  50.000000       50.000000
mean   45.840000      503.857800
std    21.688293      246.192552
min     3.000000       40.380000
25%    28.250000      382.007500
50%    48.000000      525.540000
75%    59.250000      594.942500
max    95.000000      981.010000


- __errors="raise"__ : Par défaut, cela provoque une erreur si une date invalide est rencontrée.

- __errors="ignore"__ : Ignore les erreurs et renvoie les valeurs d'origine non modifiées.

- __errors="coerce"__ : Remplace les valeurs invalides par NaT.